# Data analysis in the Amazon audit
Most of the materials in this notebook were kindly provided by Piotr Sapiezynski (www.sapiezynski.com)

### The motivation behind the analysis
Read it here
https://themarkup.org/amazons-advantage/2021/10/14/amazon-puts-its-own-brands-first-above-better-rated-products

In [1]:
# installing the packages for today (if needed)
!pip install -U pandas
!pip install -U scikit-learn
!pip install -U statsmodels

     |████████████████████████████████| 11.1 MB 2.7 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.1
    Uninstalling pandas-1.4.1:
      Successfully uninstalled pandas-1.4.1


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# this is the data from the markup audit, downloaded from their github: 
# https://github.com/the-markup/investigation-amazon-brands 

data = pd.read_csv('https://www.sapiezynski.com/cs4910/markup/data.csv')

In [4]:
data

,search_term,placed_higher,stars_delta,reviews_delta,is_shipped_by_amazon,is_sold_by_amazon,is_amazon,is_top_clicked,random_noise,asin_1,asin_2
0,#10 envelope,True,0.0,1654.0,0,2,2,0,-2.569736,B06VVLD2GL,B01D0OANU4
1,#6 envelope,True,0.1,7844.0,0,2,2,2,0.951532,B06X15WSLL,B07JNXMBSX
2,1 inch binder,False,0.0,-9383.0,0,0,-2,0,0.618294,B00A45VF2S,B01BRGTWOA
3,1% milk,False,0.0,-183.0,0,0,0,-2,0.645435,B07W5Z8SJ8,B07WC9MMPD
4,10 dollar gifts,True,0.8,75410.0,0,2,0,0,-1.075435,B00F4CEHNK,B07FCNYND8
...,...,...,...,...,...,...,...,...,...,...,...
1410,zwave hub,True,0.8,848.0,0,0,0,2,0.047647,B07D19VVTX,B077Y939JQ
1411,zxzy womens hoodies,True,-0.1,6.0,0,0,0,0,0.009533,B08F7RFZGP,B08KT3BD2X
1412,zyliss knives,True,0.0,3010.0,0,0,0,0,-0.148278,B01DKEB5OC,B0753SKVZX
1413,zyrtec for kids,True,-0.2,2184.0,2,2,0,0,1.474693,B004E0QGOQ,B088X7151W


![features](https://mrkp-static-production.themarkup.org/graphics/amazon-methodology-product-comparison/1634143723511/assets/product-pairs-subtraction-desktop.png)

In [295]:
columns = ['stars_delta', 'reviews_delta', 'is_amazon',
        'is_shipped_by_amazon', 'is_sold_by_amazon',
        'is_top_clicked', 'placed_higher']

In [296]:
dataset = data[columns]

In [297]:
train, test = train_test_split(dataset, test_size=0.2, stratify=dataset['placed_higher'])

In [298]:
features = ['is_amazon',
        'is_shipped_by_amazon', 'is_sold_by_amazon',
        'is_top_clicked', 'stars_delta', 'reviews_delta', ]

In [299]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500, max_depth=3, max_features = 6)
clf.fit(train[features], train['placed_higher'])


RandomForestClassifier(max_depth=3, max_features=6, n_estimators=500)

In [304]:
# let's see how well it does in predicting the test set
# 1. predict the test data
y_pred = clf.predict(test[features])

# 2. calculate accuracy, i.e. the fraction of time that the prediction was correct out of all predictions
(y_pred == test['placed_higher']).sum()/len(y_pred)

0.6855123674911661

In [305]:
for feature, importance in zip(features, clf.feature_importances_):
    print(feature, importance)

is_amazon 0.9032928487280287
is_shipped_by_amazon 0.007487632851518247
is_sold_by_amazon 0.002829947430288105
is_top_clicked 0.002445399880340501
stars_delta 0.019225107462222012
reviews_delta 0.06471906364760226


![features](https://www.sapiezynski.com/cs4910/markup/importances.png)

So clearly, `is_amazon` is the most important, but what exactly does it mean, how does it translate the the probability of being the top result? RandomForests won't tell us... But logistic regression can.

First, quick reminder on regression. 

**Linear** regression has the following formula:

$$ y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... $$

This means that:
* when all variables are equal to 0, the outcome variable $y$ is equal to $\beta_0$, the intercept.
* a **unit** change in $x_1$ corresponds to $y$ changing by $\beta_1$ 

It might not be a powerful model, but it offers a clear interpretation/explanation in regression problems.

**Logistic** regression is mostly used for binary classification and it looks similar to the linear regression, but now the outcome variable is the logarithm of odds of success.

$$ ln(\frac{\pi}{1-\pi}) = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... $$

where $pi$ is the probability of success, so $1-\pi$ is the probability of failure.

Then, a unit change in $x_1$ corresponds to $\beta_1$ change in log odd ratios... but that is not intuitive at all. Let's rewrite it to calculate the probability directly:

$$\begin{align}
ln(\frac{\pi}{1-\pi}) &= \beta_0 + \beta_1x_1 + \beta_2x_2 + ... \\
\frac{\pi}{1-\pi} &= e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...} \\
{\pi} &= ({1-\pi})e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...} \\
{\pi} &= e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}-\pi e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}\\
{\pi}(1+e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}) &= e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...} \\
{\pi} &= \frac{e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}}{1+e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}}
\end{align}$$

So, while we cannot directly say from that a unit change of $x_1$ corresponds to a certain change in the probability of the outcome, we can just plug zeros and ones into the equation above and compute the changes.

Let's load the data from scratch and do some cleaning (I'm dividing by two, because The Markup left the values of -2, 0, 2 so they never have a "unit" change. If we divide them by two, the unit change actually is meaningful).

In [273]:
data = pd.read_csv('https://www.sapiezynski.com/cs4910/markup/data.csv')
data['is_shipped_by_amazon'] /= 2
data['is_sold_by_amazon'] /= 2
data['is_amazon'] /= 2
data['is_top_clicked'] /= 2
dataset = data[columns]

train, test = train_test_split(dataset, test_size=0.2, stratify=dataset['placed_higher'])


Let's start with a model that only has the intercept (constant) and `is_amazon` as features.

In [281]:
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

# adding the intercept as a variable
train = add_constant(train)

# selecting which features to train on
features = ['const', 'is_amazon']

# training a logistic regression model. First the dependent (outcome variable), then the features
log_reg = sm.Logit(train['placed_higher'], train[features]).fit()

Optimization terminated successfully.
         Current function value: 0.494906
         Iterations 6


In [282]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          placed_higher   No. Observations:                 1132
Model:                          Logit   Df Residuals:                     1130
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.2859
Time:                        11:20:43   Log-Likelihood:                -560.23
converged:                       True   LL-Null:                       -784.58
Covariance Type:            nonrobust   LLR p-value:                 1.391e-99
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0883      0.073      1.208      0.227      -0.055       0.232
is_amazon      2.4883      0.163     15.278      0.000       2.169       2.807
==============================================================================
"""

We will mostly be looking at the bottom table.

`const` (the intercept) translates to the probability if all other variables are equal to 0

$$ p = \frac{e^{\beta_{const}}}{1+e^{\beta_{const}}} $$

In [283]:
import numpy as np

# the beta coefficients are stored in the .params list, with the constant being first
# get the constant out of the fit model:
const = log_reg.params[0]

np.e**const/(1+np.e**const) 


0.5220557281748975

0.5220557281748975

This should be very close to 50%. In our case if all variables are zero, it means that either:
1. The products are both from amazon
2. The products are both not from amazon

We don't have a way to differentiate them, so it's a coin toss (50%)

The other coefficients describe the change in odds that corresponds to a unit change in the variable.
* if the coefficient is positive, it means that an **increase** of variable corresponds to an **increase** in the odds of the positive outcome
* if the coefficient is positive, it means that an **increase** of variable corresponds to an **decrease** in the odds of the positive outcome


So what happens when `is_amazon` is equal to 1, i.e. one product is an amazon product and the other isn't?
By looking at the `coef` we already know it's going to go up (because the coefficient is positive). But by How much? Let's calculate:


$$ p = \frac{e^{\beta_{const} + \beta_{is\_amazon}}}{1+e^{\beta_{const} + \beta_{is\_amazon}}} $$

In [284]:
numerator = np.e**(log_reg.params[0] + log_reg.params[1])
print(numerator/(1+numerator))

0.9293369710808036


That means that in this dataset 93\% of the time we have two products where one is from amazon and the other isn't, it's the amazon product that will be on the first place.

Solved? Well no, because maybe this is just because amazon products are better. Let's include the star difference in the analysis to **control** for this:

In [285]:
features = ['const', 'is_amazon','stars_delta', ]
log_reg = sm.Logit(train['placed_higher'], train[features]).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.494329
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          placed_higher   No. Observations:                 1132
Model:                          Logit   Df Residuals:                     1129
Method:                           MLE   Df Model:                            2
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.2868
Time:                        11:21:33   Log-Likelihood:                -559.58
converged:                       True   LL-Null:                       -784.58
Covariance Type:            nonrobust   LLR p-value:                 1.925e-98
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0867      0.073      1.185      0.236      -0.057       0.230
is_amazon       2.4786      0.163     15.202      0.000       2.159       2.798
stars_delta    -0.2737      0.241     -1.138      0.255      -0.745       0.198
===============================================================================
"""

Now notice the last two columns - that's the 95% confidence interval. If one value is positive, and the other negative, the interval includes 0, which means we can't really tell if the effect is positive, or negative - it's not significant. Turns out, controling for the star rating doesn't change anything.

Let's add the review delta:

In [286]:
features = ['const', 'is_amazon','stars_delta', 'reviews_delta']
log_reg = sm.Logit(train['placed_higher'], train[features]).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.493737
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          placed_higher   No. Observations:                 1132
Model:                          Logit   Df Residuals:                     1128
Method:                           MLE   Df Model:                            3
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.2876
Time:                        11:21:57   Log-Likelihood:                -558.91
converged:                       True   LL-Null:                       -784.58
Covariance Type:            nonrobust   LLR p-value:                 1.673e-97
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0854      0.073      1.167      0.243      -0.058       0.229
is_amazon         2.4875      0.163     15.216      0.000       2.167       2.808
stars_delta      -0.2588      0.241     -1.075      0.282      -0.730       0.213
reviews_delta -1.353e-06   1.17e-06     -1.157      0.247   -3.64e-06    9.39e-07
=================================================================================
"""

Same story, so let's include the rest of our variables.

In [287]:
features = ['const', 'is_amazon','stars_delta', 'reviews_delta',
        'is_shipped_by_amazon', 'is_sold_by_amazon',
        'is_top_clicked']
log_reg = sm.Logit(train['placed_higher'], train[features]).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.485827
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          placed_higher   No. Observations:                 1132
Model:                          Logit   Df Residuals:                     1125
Method:                           MLE   Df Model:                            6
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.2990
Time:                        11:22:17   Log-Likelihood:                -549.96
converged:                       True   LL-Null:                       -784.58
Covariance Type:            nonrobust   LLR p-value:                 3.532e-98
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.1059      0.074      1.428      0.153      -0.039       0.251
is_amazon                2.2388      0.174     12.889      0.000       1.898       2.579
stars_delta             -0.3405      0.246     -1.384      0.166      -0.823       0.142
reviews_delta        -1.501e-06   1.17e-06     -1.283      0.200   -3.79e-06    7.93e-07
is_shipped_by_amazon    -0.2945      0.240     -1.225      0.221      -0.766       0.177
is_sold_by_amazon        0.7100      0.172      4.124      0.000       0.373       1.047
is_top_clicked           0.0046      0.144      0.032      0.974      -0.277       0.286
========================================================================================
"""

The only other significant variable is `is_sold_by_amazon`. How do we interpret its value?

For exaple, both products are from amazon or are not from amazon, but one is sold by amazon, so `is_amazon = 0` and `is_sold_by_amazon = 1`



In [288]:
numerator = np.e**(log_reg.params[0] + log_reg.params[1] * 0 + log_reg.params[5] * 1)
print(numerator/(1+numerator))

0.6933580136415483


That's the probability the product sold by amazon will be first.

For another example, first product is from amazon and sold by amazon, the other is neither:


In [289]:
numerator = np.e**(log_reg.params[0] + log_reg.params[1] * 1 + log_reg.params[5] * 1)
print(numerator/(1+numerator))

0.9549840944621973


That's the probability the product sold by amazon will be first - it's slightly higher than when the first product is from amazon and the other isn't but we don't know who sells it.


What about the predictive performance? The Markup uses RandomForests with hyperparameter tuning to get the best prediction and ends up with 73.2% accuracy

In [290]:
test = add_constant(test)
y_pred = log_reg.predict(test[features])

In [291]:
((y_pred > 0.5) == test['placed_higher']).mean()

0.696113074204947

In [292]:
train.shape

(1132, 8)

In [271]:
train

,const,stars_delta,reviews_delta,is_amazon,is_shipped_by_amazon,is_sold_by_amazon,is_top_clicked,placed_higher
66,1.0,0.2,11367.0,1.0,0.0,1.0,0.0,True
706,1.0,0.1,-17591.0,-1.0,0.0,0.0,0.0,False
428,1.0,0.0,1159.0,0.0,1.0,0.0,1.0,False
276,1.0,-0.3,20532.0,0.0,0.0,0.0,-1.0,False
675,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...
716,1.0,-0.8,-310.0,0.0,-1.0,-1.0,0.0,True
1135,1.0,-0.2,-7298.0,0.0,0.0,0.0,-1.0,True
534,1.0,0.1,3515.0,0.0,0.0,0.0,0.0,True
193,1.0,-0.1,-484236.0,-1.0,0.0,-1.0,-1.0,True


In [163]:
test

,const,stars_delta,reviews_delta,is_amazon,is_shipped_by_amazon,is_sold_by_amazon,is_top_clicked,placed_higher
283,1.0,0.1,0.026419,-1.0,0.0,0.0,0.0,False
925,1.0,-0.1,0.047067,0.0,0.0,0.0,0.0,True
394,1.0,-0.2,-0.011249,0.0,0.0,0.0,0.0,True
442,1.0,0.0,-0.048170,0.0,0.0,0.0,0.0,True
467,1.0,0.4,0.236989,0.0,1.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...
561,1.0,0.1,-0.610218,0.0,0.0,0.0,1.0,False
1335,1.0,-0.2,0.152138,-1.0,0.0,-1.0,0.0,False
1154,1.0,0.1,0.400056,0.0,0.0,1.0,0.0,True
85,1.0,0.1,0.006553,0.0,0.0,0.0,0.0,False


That's not any worse than random forest - we can an equally good fit but we can actually interpret just how important the different features are!

The problem that we looked at so far is defined as follows: given the top two items, which one is going to be be placed in the first position?

Now, let's solve a slightly different problem formulation (not covered in the Markup write up): given characteristics of the product, how likely is it the top product on its search page?

In [355]:
# let's get the data first:
dataset = pd.read_csv('https://www.sapiezynski.com/cs4910/markup/dataset_full.csv')

In [356]:
dataset

,stars,reviews,is_amazon,is_sold_by_amazon,is_shipped_by_amazon,top_clicked,is_top
0,4.8,2120.0,0.0,1.0,1.0,0.0,1.0
1,4.7,3747.0,0.0,1.0,1.0,1.0,0.0
2,4.8,6031.0,0.0,1.0,1.0,1.0,0.0
3,4.8,7017.0,0.0,1.0,1.0,1.0,0.0
4,4.8,455.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
83898,4.7,248.0,0.0,0.0,1.0,0.0,0.0
83899,4.8,557.0,0.0,0.0,1.0,0.0,0.0
83900,4.7,214.0,0.0,0.0,0.0,1.0,0.0
83901,4.8,451.0,0.0,0.0,1.0,1.0,0.0


This dataset is an aggregation of all first search result pages from the Markup audit that had any amazon products on them. The sponsored results are removed.

In this dataset each row is a product with its star rating, the number of reviews, the binary indicators of whether it's an amazon product, it's sold or shipped by amazon, whether it was among the most clicked products, and the outcome variable - whether it was the top product in its search results page.

We're looking at at most 24 first results, so on average only one in 24 rows is the top placed product:


In [357]:
dataset['is_top'].mean()

0.04551684683503569

In [358]:
# the lowest star rating is 1, not 0, let's adjust the rating such that it starts at 0:
dataset['stars'] -= 1

<div style="background-color:lightblue; padding:20px">
<h5>Excercise 1: Interpreting the meaning of logistic regression coefficients</h5>
In this exercise you will interpret the values of the coefficients of a logistic regression model that tries to predict whether a product was the top product on its search page.

In particular, please answer these questions:
1. What is the probability that a hypothetical product with the lowest rating, no reviews, not sold/shipped/produced by amazon, and not among the top clicked products will be placed at the top?
1. Increase in which variables corresponds to a higher probability of being placed at the top?
1. How do you interpret the `stars` coefficient and its statistical significance?
1. If a product receives one more review without changing any other of its characteristics, how does it affect its chances to placed at the top? 
    * How about 1000000 more reviews?
1. What is the probability that a five-star product with 1000 reviews, produced, sold, and shipped by amazon that was among the top clicked results will be placed on the top of the result list? 
    * Is this number higher, or lower than you expected? Does it change how you interpret the results of the model for only the top two positions? Why?

</div>

In [359]:
dataset = add_constant(dataset)
cols =['const', 'stars', 'reviews', 'is_amazon', 'is_sold_by_amazon', 'is_shipped_by_amazon', 'top_clicked']
log_reg = sm.Logit(dataset['is_top'],
                   dataset[cols]).fit()

Optimization terminated successfully.
         Current function value: 0.180478
         Iterations 7


In [360]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 is_top   No. Observations:                83903
Model:                          Logit   Df Residuals:                    83896
Method:                           MLE   Df Model:                            6
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                 0.02496
Time:                        12:22:50   Log-Likelihood:                -15143.
converged:                       True   LL-Null:                       -15530.
Covariance Type:            nonrobust   LLR p-value:                3.631e-164
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -3.4286      0.221    -15.494      0.000      -3.862      -2.995
stars                   -0.0472      0.061     -0.770      0.441      -0.167       0.073
reviews               1.284e-06   4.15e-07      3.092      0.002     4.7e-07     2.1e-06
is_amazon                1.1472      0.047     24.576      0.000       1.056       1.239
is_sold_by_amazon       -0.1224      0.039     -3.171      0.002      -0.198      -0.047
is_shipped_by_amazon     0.2063      0.068      3.040      0.002       0.073       0.339
top_clicked              0.4318      0.035     12.174      0.000       0.362       0.501
========================================================================================
"""

In [361]:
# What is the probability that a hypothetical product with the lowest rating, no reviews, 
# not sold/shipped/produced by amazon, and not among the top clicked products will be placed at the top?
np.e**(log_reg.params[0])/(1+ np.e**(log_reg.params[0]))

0.03141344729908379

- Increase in which variables corresponds to a higher probability of being placed at the top?
>reviews, is_amazon, is_shipped_by_amazon, top_clicked (all ones that have positive coefs and are significant)

- How do you interpret the stars coefficient and its statistical significance?
> It is not statistically significant, so it appears not to have an effect on the placement.
> its value is negative, so if it was significant it would indicate that higher rated items are less likely to be on top

- If a product with no reviews receives its first one without changing any other of its characteristics, how does it affect its chances to placed at the top? How about when it receives 1000000 reviews instead?

In [362]:
p0 = np.e**(log_reg.params[0])/(1+ np.e**(log_reg.params[0]))
p1 = np.e**(log_reg.params[0] + log_reg.params[2])/(1+ np.e**(log_reg.params[0]+log_reg.params[2]))
print(p1-p0)

3.907903880356889e-08


In [363]:
p2 = np.e**(log_reg.params[0] + log_reg.params[2] * 1000000)/(1+ np.e**(log_reg.params[0]+log_reg.params[2]*1000000))

In [364]:
print(p2-p0)

0.07345786543985264


In [365]:
p0

0.03141344729908379

In [366]:
p2

0.10487131273893643

> The probability of being placed at the top grows by 7 percentage points from 3% to 10%

- What is the probability that a five-star product with 100000 reviews, produced, sold, and shipped by amazon that was among the top clicked results will be placed on the top of the result list? 

In [367]:
numerator = np.e**(log_reg.params[0]\
                  + log_reg.params[1]*4\
                  + log_reg.params[2]*100000\
                  + log_reg.params[3]\
                  + log_reg.params[4]\
                  + log_reg.params[5]\
                  + log_reg.params[6])

In [368]:
numerator/(1+ numerator)

0.13871729855646864

- Is this number higher, or lower than you expected? Does it change how you interpret the results of the model for only the top two positions? Why? 

> Any answer beyond "it's what I expected, it doesn't change anything" goes. 
>
> To me personally it's surprisingly low - given the results from the markup I would naively expect that being an amazon product nearly guarantees the first spot but of course it doesn't - there are likely multiple amazon products on the front page and only one will make it to the top, and it's also not always that one is at the top.